purpose of this notebook is to hel pprepare data of varying lengths
- shorter length titles
- longer length titles

In [ ]:
import pandas as pd
import numpy as np
import pickle
import sqlite3
import os
from tqdm import tqdm
from urllib.parse import urlparse
import random
import hashlib
import time

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from model2vec import StaticModel

In [ ]:
EN_MSMARCO_DATA_PATH = "/Users/cgopal/Downloads/web_data/msmarco_english.csv"

In [ ]:
english_data = pd.read_csv(EN_MSMARCO_DATA_PATH, header=None)
english_data.columns = ['docid', 'url', 'title', 'body']
english_data['lang'] = 'en'
english_data['topic'] = ''
english_data = english_data.loc[~english_data['title'].isna()].reset_index(drop=True)
english_data['description'] = english_data['body'].fillna('').apply(lambda body: body[:300])
english_data = english_data.loc[english_data['title'].apply(len) <= 100].reset_index(drop=True)
print(len(english_data))
english_data

In [ ]:
static_model = StaticModel.from_pretrained("minishlab/potion-retrieval-32M")

In [ ]:
english_data['title'].apply(len).describe()

In [ ]:
english_data_short_titles = english_data.loc[english_data['title'].apply(len) < 30 ].reset_index(drop=True)
english_data_short_titles['description'] = ""
english_data_short_titles['text'] = english_data_short_titles['title']
english_data_short_titles

In [ ]:
english_data_long_titles = english_data.loc[english_data['title'].apply(len) >= 44 ].reset_index(drop=True)

In [ ]:
english_data_long_titles['text'] = (english_data_long_titles['title'].fillna("") + " " + english_data_long_titles['description'].fillna("")).str.strip()
# (english_data_long_titles['title'] + " " + english_data_long_titles['description']).apply(len).describe()

In [ ]:

english_data_short_titles_embeddings = static_model.encode(english_data_short_titles['text'].values.tolist())
english_data_short_titles_embeddings.shape

In [ ]:

english_data_long_titles_embeddings = static_model.encode(english_data_long_titles['text'].values.tolist())
english_data_long_titles_embeddings.shape

In [ ]:
short_queries = [
    "healthy rice",
    "endodontist",
    "arsenal",
    "gavaskar border trophy",
    "grand slam champion",
    "capital gain",
    "credit limit",
    "annuity",
    "physiology",
    "myopia",
    "amyloidosis",
    "alopecia",
    "dermatitis",
    "coffee shop",
    "ball drop"
]

short_queries_embeddings = static_model.encode(short_queries)
short_queries_embeddings.shape

In [ ]:
long_queries = [
    "alternative medicine to manage ringing in my ear",
    "solutions for noisy neighbors",
    "how to train a cat to use the litter box",
    "how to make your dog loss weight",
    "what are the best foods for dogs",
    "how to keep your kids safe online",
    "some recommendations for vegan recipes for kids",
    "best places to visit in new york in summer",
    "how much does it cost to go to niagara falls",
    "how to take screenshot on windows 11",
    "what are some of the symptoms of high blood pressure",
    "what are some treatment options for stroke",
    "organic dog food for sensitive stomachs",
    "best budget-friendly family vacation spots in California",
    "what are some popular open source projects",
]

long_queries_embeddings = static_model.encode(long_queries)
long_queries_embeddings.shape

In [ ]:
# cosine_similarity(static_model.encode(["healthy rice"]), static_model.encode(["Broadcast tower KT8"]))

In [ ]:
# cosine_similarity(static_model.encode(["healthy rice"]), english_data_short_titles_embeddings).argmax()

In [ ]:
## select from english_data_short_titles
## short Q - short D


def compute_cosine_similarities(query_embeddings, docs_embeddings):
    return cosine_similarity(query_embeddings, docs_embeddings)

def get_positive_and_negative_examples(sim_scores,
                                       docs_df,
                                       top_k = 2,
                                       text_col='text',
                                       neg_size=500
                                      ):
    
    top_items = np.argsort(-sim_scores, axis=1)[:, :top_k]  # shape (n_queries, 2)
    
    # Combine top-2 indices from each query to get positive samples
    top_items_combined = top_items.flatten()
    # print(f"top_items_combined.shape = {top_items_combined.shape}")
    docs = docs_df[text_col].values.tolist()
    # print(f"len(docs) = {len(docs)}")
    # print(f"top_items_combined = {top_items_combined}")
    positive_examples = docs_df.iloc[top_items_combined]
    print(f"#positive examples: {len(positive_examples)}")
    
    n_queries, n_docs = sim_scores.shape
    
    # Get all document indices
    all_doc_indices = np.arange(n_docs)
    
    # For each query, exclude top-k indices and sample a "hard negative" or "random negative"
    negative_indices = []
    
    for i in range(n_queries):
        pos_indices = set(top_items[i])
        neg_candidates = list(set(all_doc_indices) - pos_indices)
        
        # Choose 1 negative index per query randomly (or more, depending on use case)
        neg_idx = np.random.choice(neg_candidates, size=neg_size, replace=False)
        # print(neg_idx)
        negative_indices.append(neg_idx)
    
    # Retrieve negative examples
    flat_negative_indices = np.concatenate(negative_indices)
    
    # Now use iloc
    negative_examples = docs_df.iloc[flat_negative_indices]
    
    print(f"#negative_examples: {len(negative_examples)}")
    
    return pd.concat([
        positive_examples.reset_index(drop=True),
        negative_examples.reset_index(drop=True)
    ]).drop(text_col, axis=1)

In [ ]:
shortQ_shortD_sim = compute_cosine_similarities(query_embeddings=short_queries_embeddings, 
                                                docs_embeddings=english_data_short_titles_embeddings)


shortQ_shortD_df = get_positive_and_negative_examples(sim_scores=shortQ_shortD_sim,
                                                      docs_df=english_data_short_titles,
                                                      top_k = 2,
                                                      text_col='text',
                                                      neg_size=500
                                                      )

In [ ]:
shortQ_shortD_df

In [ ]:
shortQ_longD_sim = compute_cosine_similarities(query_embeddings=short_queries_embeddings, 
                                                docs_embeddings=english_data_long_titles_embeddings)

shortQ_longD_df = get_positive_and_negative_examples(sim_scores=shortQ_longD_sim,
                                                     docs_df=english_data_long_titles,
                                                     top_k = 2,
                                                     text_col='text',
                                                     neg_size=600
                                                     )

shortQ_longD_df

In [ ]:
## long Q - short D

longQ_shortD_sim = compute_cosine_similarities(query_embeddings=long_queries_embeddings, 
                                               docs_embeddings=english_data_short_titles_embeddings)

longQ_shortD_df = get_positive_and_negative_examples(sim_scores=longQ_shortD_sim,
                                                     docs_df=english_data_short_titles,
                                                     top_k = 2,
                                                     text_col='text',
                                                     neg_size=500
                                                     )

longQ_shortD_df

In [ ]:
## long Q - long D

longQ_longD_sim = compute_cosine_similarities(query_embeddings=long_queries_embeddings, 
                                               docs_embeddings=english_data_long_titles_embeddings)

longQ_longD_df = get_positive_and_negative_examples(sim_scores=longQ_longD_sim,
                                                     docs_df=english_data_long_titles,
                                                     top_k = 2,
                                                     text_col='text',
                                                     neg_size=600
                                                     )

longQ_longD_df

In [ ]:
def generate_url_hash(url):
    hash_object = hashlib.md5(url.encode("utf-8"))  
    url_hash = int(hash_object.hexdigest(), 16) % (10 ** 14) 
    return url_hash


def generate_last_visit_date(index):
    base_time = int(time.time() * 1_000_000)  # Current time in microseconds
    return base_time + index * 2_000  # Increment by 2000 microseconds (2ms) per row

In [ ]:
profiles = {'longQ_longD_df': longQ_longD_df,
            'longQ_shortD_df': longQ_shortD_df,
            'shortQ_shortD_df': shortQ_shortD_df,
            'shortQ_longD_df': shortQ_longD_df
           }

profiles_generated = dict()
for profile, df in profiles.items():
    frecency = [random.randint(10,500) for _ in range(len(df))]
    df['frecency'] = frecency
    df['url_hash'] = df['url'].apply(generate_url_hash)
    df["last_visit_date"] = [generate_last_visit_date(i) for i in range(len(df))]
    # profiles['profile'] = df
    profiles_generated[profile] = df


In [ ]:
profiles.keys()

In [ ]:
profiles_generated['longQ_longD_df']

In [ ]:
def prepare_synthetic_profile(profile_data):
    print("original = ", len(profile_data))
    # profile_data = profile_data.drop_duplicates(['url'], keep='first').reset_index(drop=True)
    # profile_data = profile_data.loc[~profile_data['url'].isna()].reset_index(drop=True)
    profile_data = profile_data.drop_duplicates(['url_hash']).reset_index(drop=True)
    print("after cleaned up = ", len(profile_data))
    profile_data['domain'] = profile_data['url'].apply(lambda x: urlparse(x).netloc.split(':')[0])
    profile_data['combined_text'] = profile_data['title'] + " " +   profile_data['description'].fillna("")
    return profile_data

def get_connection_to_places():
    return sqlite3.connect("../data/places.sqlite")

def get_personal_history(row_limit, sql_conn):
    input_data = pd.read_sql(f"""
        SELECT *
        FROM moz_places p
        WHERE p.title NOTNULL
        AND url not like '%google.com/search?%'
        ORDER BY frecency DESC
        LIMIT {row_limit}; 
        """, sql_conn)
    
    # print(len(input_data))
    return input_data


def prepare_dummy_profile(input_data, samples_df):
    input_data['url'] = samples_df['url'].values.tolist()
    input_data['title'] = samples_df['title'].values.tolist()
    input_data['url_hash'] = samples_df['url_hash'].values.tolist()
    input_data['description'] = samples_df['description'].values.tolist()
    ## Fill ~75% of description as null based on some eda
    # input_data['description'] = samples_df['description'].apply(
    #     lambda desc: None if np.random.choice([True, False], p=[0.75, 0.25]) else desc
    # ).values.tolist()
    input_data['rev_host'] = samples_df['domain'].apply(lambda host: host[::-1]).values.tolist()
    input_data['preview_image_url'] = None
    return input_data

In [ ]:

def generate_dummy_profile(profile_data):
    profile_data = prepare_synthetic_profile(profile_data)
    row_limit = len(profile_data)
    sql_conn = get_connection_to_places()
    input_data = get_personal_history(row_limit, sql_conn)
    
    samples_df = profile_data.reset_index(drop=True)
    # print(len(samples_df))
    return prepare_dummy_profile(input_data, samples_df)

In [ ]:
def prepare_sqlite_db(df, places_db_path):
    # Connect to SQLite database
    conn = sqlite3.connect(places_db_path)
    
    # Write DataFrame to SQLite database
    df.to_sql("moz_places", conn, if_exists="replace", index=False)  # Modify table name as needed
    
    # Close connection
    conn.close()

In [ ]:
os.makedirs("../data/profiles/test", exist_ok=True)

for profile, profile_data in profiles_generated.items():
    profile_data_enriched = generate_dummy_profile(profile_data)
    print(profile_data_enriched.columns)
    profile_data_enriched.to_json(f"../data/profiles/test/profile_{profile}.json", orient="records", indent=2)


In [ ]:
!ls ../data/profiles/test/profile_*